# Classes

In [ ]:
class Dog:

    kind = 'canine'         # class variable shared by all instances

    def __init__(self, name):
        self.name = name    # instance variable unique to each instance

d = Dog('Fido')
e = Dog('Buddy')
d.kind 

'canine'

## Abstract Base Classes

In [ ]:
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

class Circle(Shape):
    def __init__(self, r):
        self.r = r
    def area(self):
        from math import pi
        return pi * self.r ** 2

c = Circle(5)
print(c.area())

## Generic Classes

In [ ]:
from typing import Generic, TypeVar
T = TypeVar("T")

class Box(Generic[T]):
    def __init__(self, value: T): self.value = value
    def __repr__(self): return f"Box({self.value!r})"

int_box = Box
str_box = Box[str]("hello")
print(int_box, str_box)

## Inheritance

In [ ]:
## Inheritance
class Animal:
    def speak(self):
        return "Some sound"

class Dog(Animal):
    def speak(self):
        return "Woof!"

class Cat(Animal):
    def speak(self):
        return "Meow!"

animals = [Dog(), Cat(), Animal()]
for a in animals:
    print(a.speak())

### super()

In [ ]:
## super() for Extending Behavior
class Person:
    def __init__(self, name):
        self.name = name

class Employee(Person):
    def __init__(self, name, job):
        super().__init__(name)
        self.job = job

emp = Employee("Alice", "Engineer")
print(emp.name, emp.job)


## Class Variables

### Private & Protected

In [ ]:
## Private and Protected Conventions
class BankAccount:
    def __init__(self, balance):
        self._balance = balance      # protected by convention
        self.__pin = 1234            # name-mangled private var

    def deposit(self, amount):
        self._balance += amount

    def get_balance(self):
        return self._balance

acc = BankAccount(100)
acc.deposit(50)
print(acc.get_balance())      # 150
# print(acc.__pin)            # AttributeError
print(acc._BankAccount__pin)  # Access via name mangling (not recommended)


## Methods

### Static

In [ ]:
class MathUtils:
    factor = 10

    @classmethod
    def scale(cls, x):
        return x * cls.factor

    @staticmethod
    def add(a, b):
        return a + b

print(MathUtils.scale(3))   # 30
print(MathUtils.add(5, 7))  # 12

### \_\_repr\_\_ & \_\_str\_\_

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __repr__(self):
        return f"Point({self.x!r}, {self.y!r})"

    def __str__(self):
        return f"({self.x}, {self.y})"

p = Point(1, 2)
print(p)         # (1, 2)
print(repr(p))   # Point(1, 2)


### \_\_eq\_\_ & \_\_add\_\_

In [ ]:
class Vector:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)

    def __repr__(self):
        return f"Vector({self.x}, {self.y})"

v1 = Vector(2, 3)
v2 = Vector(1, 4)
print(v1 + v2)       # Vector(3, 7)
print(v1 == v2)      # False

### \_\_enter\_\_ & \_\_exit\_\_

In [ ]:
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        self.file = open(self.filename, self.mode)
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.file.close()

# with FileManager("test.txt", "w") as f:
#     f.write("Hello world!")

### \_\_call\_\_

In [ ]:
## Callable Objects (__call__)
class Accumulator:
    def __init__(self, start=0): self.total = start
    def __call__(self, x):       self.total += x; return self.total

acc = Accumulator()
print(acc(5))   # 5
print(acc(7))   # 12

### Getters, Setters, Properties

In [ ]:
class Temperature:
    def __init__(self, celsius):
        self._celsius = celsius

    @property
    def celsius(self):
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Temperature below absolute zero!")
        self._celsius = value

    @property
    def fahrenheit(self):
        return self._celsius * 9/5 + 32

t = Temperature(25)
print(t.fahrenheit)  # 77.0
t.celsius = 0
print(t.fahrenheit)  # 32.0

## Dataclasses

In [ ]:
from dataclasses import dataclass

@dataclass
class Employee:
    name: str
    dept: str
    salary: int

### Dataclass features

In [ ]:
## dataclass features (auto repr, eq, ordering)
from dataclasses import dataclass

@dataclass(order=True)
class Student:
    grade: int
    name: str

s1 = Student(90, "Alice")
s2 = Student(85, "Bob")
print(s1 > s2)  # True, since grade compared first


## Iterators

- Behind the scenes, the for statement calls iter() on the container object. 
- The function returns an iterator object that defines the method __next__() which accesses elements in the container one at a time. 
- When there are no more elements, __next__() raises a StopIteration exception which tells the for loop to terminate.

In [2]:
s = 'abc'
it = iter(s)
print(next(it))
print(next(it))
print(next(it))
print(next(it))

a
b
c


StopIteration: 

In [3]:
class Reverse:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]
    
rev = Reverse('spam')
for char in rev:
    print(char)

m
a
p
s


## Generators

- simple and powerful tool for creating iterators
- written like regular functions but use the yield statement whenever they want to return data
- Each time next() is called on it, the generator resumes where it left off (it remembers all the data values and which statement was last executed)

In [ ]:
from functools import wraps

def retries(n: int):
    def deco(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            last = None
            for _ in range(n):
                try:
                    return fn(*args, **kwargs)
                except Exception as e:
                    last = e
            # only raise if always failed
            raise last
        return wrapper
    return deco

@retries(3)
def maybe_fail(x: int):
    if x < 0:
        raise ValueError("bad")
    return x

print(maybe_fail(5))   # 5
# print(maybe_fail(-1))  # would raise after 3 tries

### Class-based Generators

In [ ]:
class Counter:
    def __init__(self, low, high):
        self.low, self.high = low, high

    def __iter__(self):
        for i in range(self.low, self.high + 1):
            yield i

for i in Counter(3, 6):
    print(i)